In [6]:
!pip install git+https://github.com/deepset-ai/haystack.git@main


  Cloning https://github.com/deepset-ai/haystack.git (to revision main) to /tmp/pip-req-build-sw6lay9c
  Running command git clone --filter=blob:none --quiet https://github.com/deepset-ai/haystack.git /tmp/pip-req-build-sw6lay9c
  Resolved https://github.com/deepset-ai/haystack.git to commit 1d96e6e4af385b2ae00fd07c085abc8d6799e685
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 11.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 102.6/102.6 kB 10.8 MB/s eta 0:00:00
  Created wheel for haystack-ai: filename=haystack_ai-2.17.0rc0-py3-none-any.whl size=575587 sha256=9f71bde47d1af70ddaefea56520f4edc50c0f025a047a6603896d925e1e2851b
  Stored in directory: /tmp/pip-ephem-wheel-cache-qlee0bi8/wheels/b1/48/49/19ff23db8ee37ab13155d169607bf99ee3d13fc90877af4437
Successfully built haystack-ai


In [7]:
!pip install --upgrade pydantic


In [8]:
from haystack.document_stores.in_memory import InMemoryDocumentStore
from haystack.dataclasses import Document

document_store = InMemoryDocumentStore()


In [9]:
from datasets import load_dataset
from haystack import Document

dataset = load_dataset("bilgeyucel/seven-wonders", split="train")
docs = [Document(content = doc["content"],meta=doc["meta"]) for doc in dataset]

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/46.0 [00:00<?, ?B/s]

(…)-00000-of-00001-4077bd623d55100a.parquet:   0%|          | 0.00/119k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/151 [00:00<?, ? examples/s]

In [10]:
from haystack.components.embedders import SentenceTransformersDocumentEmbedder

doc_embedder = SentenceTransformersDocumentEmbedder(model="sentence-transformers/all-MiniLM-L6-v2")
doc_embedder.warm_up()

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [11]:
docs_with_embeddings = doc_embedder.run(docs)
document_store.write_documents(docs_with_embeddings["documents"])


Batches:   0%|          | 0/5 [00:00<?, ?it/s]

151

In [50]:
from haystack.components.embedders import SentenceTransformersTextEmbedder

text_embedder = SentenceTransformersTextEmbedder(model="sentence-transformers/all-MiniLM-L6-v2")

In [51]:
from haystack.components.retrievers.in_memory import InMemoryEmbeddingRetriever

retriever = InMemoryEmbeddingRetriever(document_store=document_store)

In [52]:
from haystack.components.builders import ChatPromptBuilder
from haystack.dataclasses import ChatMessage

template = [
    ChatMessage.from_user(
        """
        Given the following information , answer the question.

        Context:
        {% for document in documents %}
          {{document.content}}
        {%endfor%}

        Question : {{question}}
        Answer :
        """
    )
]

prompt_builder=ChatPromptBuilder(template=template)

In [60]:
import os
from haystack.components.generators.chat import OpenAIChatGenerator

if "OPENAI_API_KEY" not in os.environ:
    from google.colab import auth
    os.environ["OPENAI_API_KEY"] = input("🔐 Paste your OpenAI API key here : ").strip()

chat_generator = OpenAIChatGenerator(model="gpt-4o-mini")



In [53]:
from haystack import Pipeline
basic_rag_pipeline = Pipeline()

basic_rag_pipeline.add_component("text_embedder", text_embedder)
basic_rag_pipeline.add_component("retriever", retriever)
basic_rag_pipeline.add_component("prompt_builder", prompt_builder)
basic_rag_pipeline.add_component("llm", chat_generator)

In [39]:
basic_rag_pipeline.connect("text_embedder.embedding", "retriever.query_embedding")
basic_rag_pipeline.connect("retriever", "prompt_builder")
basic_rag_pipeline.connect("prompt_builder.prompt", "llm.messages")

🚅 Components
  - text_embedder: SentenceTransformersTextEmbedder
  - retriever: InMemoryEmbeddingRetriever
  - prompt_builder: ChatPromptBuilder
  - llm: OpenAIChatGenerator
🛤️ Connections
  - text_embedder.embedding -> retriever.query_embedding (List[float])
  - retriever.documents -> prompt_builder.documents (List[Document])
  - prompt_builder.prompt -> llm.messages (List[ChatMessage])

In [40]:
question = "What does Rhodes Statue look like?"

response = basic_rag_pipeline.run({"text_embedder":{"text":question},"prompt_builder":{"question":question}})
print(response["llm"]["replies"][0].text)

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

The Colossus of Rhodes was a massive statue of the Greek sun-god Helios, standing approximately 70 cubits, or about 33 meters (108 feet) tall. Although the exact appearance of the statue is not definitively known, ancient accounts provide a general description. It was constructed with iron tie bars and brass plates to form its skin. The statue is believed to have had curly hair with evenly spaced spikes, resembling bronze or silver flames radiating from the head, similar to images found on contemporary Rhodian coins.

The interior of the statue was filled with stone blocks, and it stood on a 15-meter-high (49-foot) white marble pedestal near the entrance to Rhodes harbor. The statue was an impressive sight, representing freedom and independence, and was meant to celebrate Rhodes' successful defense against an attack by Demetrius Poliorcetes. Its posture is often imagined as heroic and dynamic, possibly suggesting a pose similar to that of Helios shielding his eyes from the sun. The ove